# Chapter 1

In [2]:
#Changing the working directory to the root
%cd ../..

c:\Users\dioni\Documents\GitHub


In [90]:
import numpy as np
import matplotlib.pyplot as plt
import os
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"
import timesynth as ts
import pandas as pd
np.random.seed()

In [74]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


-----

Now, we make a function for plot our differents time series

In [75]:
def plot_time_series(time, values, label, legends=None):
    if legends is not None:
        assert len(legends)==len(values)
    if isinstance(values, list):
        series_dict = {"Time": time}
        for v, l in zip(values, legends):
            series_dict[l] = v
        plot_df = pd.DataFrame(series_dict)
        plot_df = pd.melt(plot_df,id_vars="Time",var_name="ts", value_name="Value")
    else:
        series_dict = {"Time": time, "Value": values, "ts":""}
        plot_df = pd.DataFrame(series_dict)
    
    if isinstance(values, list):
        fig = px.line(plot_df, x="Time", y="Value", line_dash="ts")
    else:
        fig = px.line(plot_df, x="Time", y="Value")
    fig.update_layout(
        autosize=False,
        width=900,
        height=500,
        title={
        'text': label,
#         'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        titlefont={
            "size": 25
        },
        yaxis=dict(
            title_text="Value",
            titlefont=dict(size=12),
        ),
        xaxis=dict(
            title_text="Time",
            titlefont=dict(size=12),
        )
    )
    return fig
    
def generate_timeseries(signal, noise=None):
    time_sampler = ts.TimeSampler(stop_time=20)
    regular_time_samples = time_sampler.sample_regular_time(num_points=200)
    timeseries = ts.TimeSeries(signal_generator=signal, noise_generator=noise)
    samples, signals, errors = timeseries.sample(regular_time_samples)
    return samples, regular_time_samples, signals, errors

os.makedirs("imgs/chapter_1", exist_ok=True)

----

# White Noise

In [76]:
# Generate the time axis with sequential numbers upto 200
time = np.arange(200)
# Sample 200 hundred random values
values = np.random.randn(200)*100
fig = plot_time_series(time, values, "White Noise")
fig.write_image("imgs/chapter_1/white_noise_process.png")
fig.show()

---- 
# Red Noise

Red noise has zero mean and constant variance but is serially correlated in time. This serial correlation or redness is parameterized by a correlation coefficient r, such that:
$$x_{j+1}=r\cdot x_j + (1-r^2)^{\frac{1}{2}} \cdot w$$

where $w$ is a random sample from a white noise distribution.

In [77]:
# Setting the correlation coefficent
r=0.9

# Generate the time axis
time = np.arange(200)

# Generate white noise
white_noise = np.random.randn(200)*100

# Create Red noise by introducing correlation between sebsequent values in the white noise
values = np.zeros(200)
for i,v in enumerate(white_noise): # i is for number of vector, v is for value of vector
    if i==0:
        values[i]=v
    else:
        values[i]=r*values[i-1]+np.sqrt(1-r**2)*white_noise[i-1]
        
plot_time_series(time, values,'Red Noise')

----

# Cyclical or seasonal signals

In [79]:
#Sinusoidal Signal with Amplitude=1.5 & Frequency=0.25
signal_1 =ts.signals.Sinusoidal(amplitude=1.5, frequency=0.25)
#Sinusoidal Signal with Amplitude=1 & Frequency=0. 5
signal_2 = ts.signals.Sinusoidal(amplitude=1, frequency=0.5)

In [80]:
white_noise = ts.noise.GaussianNoise(std=0.9)

In [81]:
samples_1, regular_time_samples, signals_1, errors_1 = generate_timeseries(signal=signal_1,noise=white_noise)
samples_2, regular_time_samples, signals_2, errors_2 = generate_timeseries(signal=signal_2)
plot_time_series(regular_time_samples,[samples_1, samples_2],"Sinusoidal Waves",legends=["Amplitude = 1.5 | Frequency = 0.25",
"Amplitude = 1 | Frequency = 0.5"])

# Pseudo-periodic 

`TimeSynth` also has another signal called `PseudoPeriodic`. This is like the `Sinusoidal` class, but the frequency and amplitude itself has some stochasticity. We can see in the following code snippet that this is more realistic than the vanilla sine and cosine waves from the `Sinusoidal` class:

In [91]:
# PseudoPeriodic signal with Amplitude=1 & Frequency=0.1
signal = ts.signals.PseudoPeriodic(amplitude=1, frequency=0.25)

# Generating Timeseries
samples, regular_time_samples, signals, errors = generate_timeseries(signal=signal)

plot_time_series(regular_time_samples, samples, "Pseudo Periodic")

----

# Autoregressive signals

In [83]:
# Autoregressive signal with parameters 1.5 and -0.75
# y(t) = 1.5*y(t-1) - 0.75*y(t-2)

# Initializing TimeSampler
time_sampler = ts.TimeSampler(stop_time=20)
# Sampling regular time samples
regular_time_samples = time_sampler.sample_regular_time(num_points=500)

ar_p = ts.signals.AutoRegressive(ar_param=[1.5, -0.75])
ar_p_series = ts.TimeSeries(signal_generator=ar_p)
samples = ar_p_series.sample(regular_time_samples)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [106]:
import timesynth as ts

#Generating Pseudo Periodic Signal
pseudo_samples, regular_time_samples, _, _ = generate_timeseries(
    signal=ts.signals.PseudoPeriodic(amplitude=1,frequency=0.25), noise=ts.noise.GaussianNoise(std=0.3))

# Generating an Autoregressive Signal
ar_samples, regular_time_samples, _, _ = generate_timeseries(
    signal=ts.signals.AutoRegressive(ar_param=[0.5]))

# Combining the two signals using a mathematical equation
ts = pseudo_samples*2+ar_samples
plot_time_series(regular_time_samples, ts,"Pseudo Periodic with AutoRegression and White Noise")

-----

# Change in mean over time
This is the most popular way a non-stationary time series presents itself. If there is an upward/downward
trend in the time series, the mean across two windows of time would not be the same.
Another way non-stationarity manifests itself is in the form of seasonality. Suppose we are looking
at the time series of average temperature measurements in a month for the last 5 years. From our
experience, we know that temperature peaks during summer and falls in winter. So, when we take the
mean temperature of winter and mean temperature of summer, they will be different.
Let's generate a time series with trend and seasonality and see how it manifests, as follows:

In [111]:
import timesynth as ts
# Sinusoidal Signal with Amplitude=1 & Frequency=0.25
signal=ts.signals.Sinusoidal(amplitude=1, frequency=0.25)
# White Noise with standard deviation = 0.3
noise=ts.noise.GaussianNoise(std=0.3)
# Generate the time series
sinusoidal_samples, regular_time_samples, _, _ = generate_timeseries(signal=signal, noise=noise)
# Regular_time_samples is a linear increasing time axis and can be used as a trend
plot_time_series(regular_time_samples,sinusoidal_samples,"Sinusoidal with White Noise")

In [112]:
trend = regular_time_samples*0.4
# Combining the signal and trend
ts_2 = sinusoidal_samples+trend
plot_time_series(regular_time_samples,ts_2,"Sinusoidal with Trend and White Noise")


In [114]:
20*0.4

8.0